### Simple demonstration of the use of generic dask functions

In [ ]:
import os
import sys

import numpy

sys.path.append(os.path.join('..', '..'))

import astropy.units as u

from dask import delayed

from astropy.coordinates import SkyCoord

from arl.data.polarisation import PolarisationFrame
from arl.graphs.generic_graphs import create_generic_image_graph
from arl.graphs.dask_init import get_dask_Client
from arl.image.iterators import image_raster_iter
from arl.image.operations import qa_image
from arl.util.testing_support import create_test_image

frequency = numpy.array([1e8])

phasecentre = SkyCoord(ra=+15.0 * u.deg, dec=-35.0 * u.deg, frame='icrs', equinox='J2000')
test_image = create_test_image(frequency=frequency, phasecentre=phasecentre,
                               cellsize=0.001,
                               polarisation_frame=PolarisationFrame('stokesI'))

image_graph = delayed(create_test_image)(frequency=frequency, phasecentre=phasecentre, cellsize=0.001,
                                         polarisation_frame=PolarisationFrame('stokesI'))


Define a simple function to take the square root of an image

In [ ]:
def imagerooter(im, **kwargs):
    im.data = numpy.sqrt(numpy.abs(im.data))
    return im

root = create_generic_image_graph(imagerooter, test_image, facets=4)
root.visualize()

Run the graph directly and then via a scheduler created via Client.

In [ ]:
result = root.compute()
numpy.testing.assert_array_almost_equal_nulp(result.data**2, numpy.abs(test_image.data), 7)

c = get_dask_Client()
print(c)
future=c.compute(root)
result=future.result()
numpy.testing.assert_array_almost_equal_nulp(result.data**2, numpy.abs(test_image.data), 7)
c.shutdown()